# Data Science Project Planning: Predicting Newsletter Subscription in Minecraft Players

## Introduction
This report explores a dataset from a UBC research group studying player behavior in a Minecraft server. The goal is to predict which types of players are most likely to subscribe to a game-related newsletter based on their characteristics and playing patterns.

In [1]:
library(tidyverse)
library(repr)

options(repr.plot.width = 10, repr.plot.height = 6)

options(warn = -1)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
players <- read_csv('https://raw.githubusercontent.com/mohammadrezaebrahimi/Project_Planning_Stage_Individual-final/refs/heads/main/players.csv')
sessions <- read_csv('https://raw.githubusercontent.com/mohammadrezaebrahimi/Project_Planning_Stage_Individual-final/refs/heads/main/sessions.csv')

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
